In [16]:
#n=6, q=2, k=2
######################################################################
the_q=2
the_n=6
the_k=2
the_R = GF(the_q)['x']
the_I= the_R.ideal(x^6 + x + 1)
the_A = QuotientRing(the_R, the_I)


######################################################################
# Embedding
######################################################################
def companion_matrix(poly, space):
#Returns the companion matrix in the given space of matrices for a given polynomial poly
    coefficients = poly.coefficients(sparse=False)
    entries = []
    dimension = poly.degree()
    for i in range(dimension):
        row = [0 for j in range(dimension)]
        if i>0:
            row[i-1] = 1
        row[dimension-1] = -coefficients[i]
        entries = entries + row
    return space.matrix(entries) 

def nonsplit_embedding(compmatrix, q, B):
    # Written for n = 6
    # Returns the set of matrices of the form aI+bM+ ... +fM^5 given the compmatrix M were a, ... , f",
    # are in F_q and are not all 0",
    matrices = []
    for a,b,c,d,e,f in GF(q)^the_n:
        if a!= 0 or b!=0 or c!=0 or d!=0 or e!=0 or f!=0:
            matrices.append(B.inverse()*(a*compmatrix^0+b*compmatrix+c*compmatrix^2+d*compmatrix^3+e*compmatrix^4+f*compmatrix^5)*B)
    return matrices

#I eliminated n from the input of the function and now everything is UNHAPPY
def basis_to_change_matrix(basis, q):
    #takes in a basis of F_q^n and returns the change of basis matrix C_B->A
    # where A is the standard basis 1,x,x^2,... and B is the given basis
    entries = []
    for element in basis:
        coeffs = [0 for i in range(the_n)]
        start = element.coefficients(sparse = False)
        for i in range(len(start)):
            coeffs[i] = start[i]
        entries.append(coeffs)
    transpose = MatrixSpace(GF(q),the_n,the_n).matrix(entries)
    change_matrix = transpose.transpose()
    return change_matrix


######################################################################
#  Tests
######################################################################
def threefive_test(poly, q, B):
    # Written now just for k=2, n=6 (so poly should have degree n=6)",
    # Tests that all 2x6 matrices where the first nonzero determinant is Delta_35"
    # has an orbit which intersects the slice in one point
    mspace = MatrixSpace(GF(q),the_k,the_n)
    comp_matrix=companion_matrix(poly, MatrixSpace(GF(q),the_n,the_n))
    torus_elements = nonsplit_embedding(comp_matrix, q, B)
    for a,b,c in GF(q)^3:
        entries = [0,0,1,a,0,b,0,0,0,0,1,c]
        element = mspace.matrix(entries)
        elements_with_specific_minors = set()
        for m in torus_elements:
            result = (element*m).echelon_form()
            minors = result.minors(the_k)
            if minors[0] == 1 and minors[4]==-1 and minors[5]==1 and minors[9]==1 and minors[12]==1 and minors[14]==1:
                elements_with_specific_minors.add(result)    
        if len(elements_with_specific_minors) != 1:
            return False
    return True

def twofive_test(poly, q, B):
    # Written now just for k=2, n=6 (so poly should have degree n=6)",
    # Tests that all 2x6 matrices where the first nonzero determinant is Delta_25"
    # has an orbit which intersects the slice in one point
    mspace = MatrixSpace(GF(q),the_k,the_n)
    comp_matrix=companion_matrix(poly, MatrixSpace(GF(q),the_n,the_n))
    torus_elements = nonsplit_embedding(comp_matrix, q, B)
    for a,b,c,d in GF(q)^4:
        entries = [0,1,a,b,0,c,0,0,0,0,1,d]
        element = mspace.matrix(entries)
        elements_with_specific_minors = set()
        for m in torus_elements:
            result = (element*m).echelon_form()
            minors = result.minors(the_k)
            if minors[0] == 1 and minors[4]==-1 and minors[5]==1 and minors[9]==1 and minors[12]==1 and minors[14]==1:
                elements_with_specific_minors.add(result)    
        if len(elements_with_specific_minors) != 1:
            return False
    return True

def onefive_test(poly, q, B):
    # Written now just for k=2, n=6 (so poly should have degree n=6)",
    # Tests that all 2x6 matrices where the first nonzero determinant is Delta_25"
    # has an orbit which intersects the slice in one point
    mspace = MatrixSpace(GF(q),the_k,the_n)
    comp_matrix=companion_matrix(poly, MatrixSpace(GF(q),the_n,the_n))
    torus_elements = nonsplit_embedding(comp_matrix, q, B)
    for a,b,c,d,e in GF(q)^5:
        entries = [1,a,b,c,0,d,0,0,0,0,1,e]
        element = mspace.matrix(entries)
        elements_with_specific_minors = set()
        for m in torus_elements:
            result = (element*m).echelon_form()
            minors = result.minors(the_k)
            if minors[0] == 1 and minors[4]==-1 and minors[5]==1 and minors[9]==1 and minors[12]==1 and minors[14]==1:
                elements_with_specific_minors.add(result)    
        if len(elements_with_specific_minors) != 1:
            return False
    return True

######################################################################
# Normal elements
######################################################################
normal_elems = []
for a,b,c,d,e,f in GF(the_q)^the_n:
    if a!=0 or b!=0 or c!=0 or d!=0 or e!=0 or f!=0:
        generating_poly = a*(x^5).polynomial(GF(the_q))+b*(x^4).polynomial(GF(the_q))+c*(x^3).polynomial(GF(the_q))+d*(x^2).polynomial(GF(the_q))+e*(x^1).polynomial(GF(the_q))+f*(x^0).polynomial(GF(the_q))
        powers = [the_A(generating_poly^(the_q^i)).lift() for i in range(the_n)]
        matrix = basis_to_change_matrix(powers, the_q)
        if matrix.det() != 0:
            normal_elems.append(generating_poly)
            

for p in normal_elems:
    print("Normal:", p)
print(len(normal_elems))


######################################################################
#  Putting it all together
######################################################################

works = []
for poly in normal_elems:
    powers = [the_A(poly^(the_q^i)).lift() for i in range(the_n)]
    matrix = basis_to_change_matrix(powers,the_q)
    result35 = threefive_test((x^6+x+1).polynomial(GF(the_q)), the_q, matrix)
    result25 = twofive_test((x^6+x+1).polynomial(GF(the_q)), the_q, matrix)
    result15 = onefive_test((x^6+x+1).polynomial(GF(the_q)), the_q, matrix)
    if result15 and result25 and result35:
        print("Normal element that's working:", poly)
        print(matrix)
        print()
        works.append(poly)

print(len(works))

Normal: x^5
Normal: x^5 + x^4
Normal: x^5 + x^3
Normal: x^5 + x^4 + x^3
Normal: x^5 + x^2
Normal: x^5 + x^4 + x^3 + x^2
Normal: x^5 + x^4 + x
Normal: x^5 + x^3 + x
Normal: x^5 + x^2 + x
Normal: x^5 + x^4 + x^2 + x
Normal: x^5 + x^3 + x^2 + x
Normal: x^5 + x^4 + x^3 + x^2 + x
Normal: x^5 + 1
Normal: x^5 + x^4 + 1
Normal: x^5 + x^3 + 1
Normal: x^5 + x^4 + x^3 + 1
Normal: x^5 + x^2 + 1
Normal: x^5 + x^4 + x^3 + x^2 + 1
Normal: x^5 + x^4 + x + 1
Normal: x^5 + x^3 + x + 1
Normal: x^5 + x^2 + x + 1
Normal: x^5 + x^4 + x^2 + x + 1
Normal: x^5 + x^3 + x^2 + x + 1
Normal: x^5 + x^4 + x^3 + x^2 + x + 1
24
Normal element that's working: x^5
[0 0 0 1 0 0]
[0 0 0 1 1 0]
[0 0 1 1 1 1]
[0 0 1 1 0 0]
[0 1 1 0 0 0]
[1 1 1 1 1 1]

Normal element that's working: x^5 + x^4
[0 0 1 0 0 0]
[0 0 1 1 0 0]
[0 1 1 1 1 0]
[0 1 1 0 0 0]
[1 1 0 0 0 0]
[1 1 1 1 1 1]

Normal element that's working: x^5 + x^4 + x^3
[0 1 0 1 1 0]
[0 1 1 1 0 1]
[0 1 0 1 0 0]
[1 1 1 0 1 0]
[1 1 0 1 0 1]
[1 1 1 1 1 1]

Normal element that

In [ ]:
the_Poly = (x^6+x+1).polynomial(GF(the_q))
works = []
print("Poly:", the_Poly)
for B in GL(the_n, GF(the_q)):
    if onetwo_test(the_Poly,the_q,B):
        print("OK")
        works.append(B)
        print("B:", B)

In [17]:
#n=6, q=2, k=3
######################################################################
the_q=2
the_n=6
the_k=3
the_R = GF(the_q)['x']
the_I= the_R.ideal(x^6 + x + 1)
the_A = QuotientRing(the_R, the_I)


######################################################################
# Embedding
######################################################################
def companion_matrix(poly, space):
#Returns the companion matrix in the given space of matrices for a given polynomial poly
    coefficients = poly.coefficients(sparse=False)
    entries = []
    dimension = poly.degree()
    for i in range(dimension):
        row = [0 for j in range(dimension)]
        if i>0:
            row[i-1] = 1
        row[dimension-1] = -coefficients[i]
        entries = entries + row
    return space.matrix(entries) 

def nonsplit_embedding(compmatrix, q, B):
    # Written for n = 6
    # Returns the set of matrices of the form aI+bM+ ... +fM^5 given the compmatrix M were a, ... , f",
    # are in F_q and are not all 0",
    matrices = []
    for a,b,c,d,e,f in GF(q)^the_n:
        if a!= 0 or b!=0 or c!=0 or d!=0 or e!=0 or f!=0:
            matrices.append(B.inverse()*(a*compmatrix^0+b*compmatrix+c*compmatrix^2+d*compmatrix^3+e*compmatrix^4+f*compmatrix^5)*B)
    return matrices

#I eliminated n from the input of the function and now everything is UNHAPPY
def basis_to_change_matrix(basis, q):
    #takes in a basis of F_q^n and returns the change of basis matrix C_B->A
    # where A is the standard basis 1,x,x^2,... and B is the given basis
    entries = []
    for element in basis:
        coeffs = [0 for i in range(the_n)]
        start = element.coefficients(sparse = False)
        for i in range(len(start)):
            coeffs[i] = start[i]
        entries.append(coeffs)
    transpose = MatrixSpace(GF(q),the_n,the_n).matrix(entries)
    change_matrix = transpose.transpose()
    return change_matrix


######################################################################
#  Tests
######################################################################
def threefourfive_test(poly, q, B):
    # Written now just for k=3, n=6 (so poly should have degree n=6)",
    # Tests that all 3x6 matrices where the first nonzero determinant is Delta_345"
    # has an orbit which intersects the slice in one point
    mspace = MatrixSpace(GF(q),the_k,the_n)
    comp_matrix=companion_matrix(poly, MatrixSpace(GF(q),the_n,the_n))
    torus_elements = nonsplit_embedding(comp_matrix, q, B)
    for a,b,c in GF(q)^3:
        entries = [0,0,1,0,0,a,0,0,0,1,0,b,0,0,0,0,1,c]
        element = mspace.matrix(entries)
        elements_with_specific_minors = set()
        for m in torus_elements:
            result = (element*m).echelon_form()
            minors = result.minors(the_k)
            if minors[0] == 1 and minors[4]==-1 and minors[5]==1 and minors[9]==1 and minors[12]==1 and minors[14]==1:
                elements_with_specific_minors.add(result)    
        if len(elements_with_specific_minors) != 1:
            return False
    return True


def twofourfive_test(poly, q, B):
    # Written now just for k=3, n=6 (so poly should have degree n=6)",
    # Tests that all 3x6 matrices where the first nonzero determinant is Delta_245"
    # has an orbit which intersects the slice in one point
    mspace = MatrixSpace(GF(q),the_k,the_n)
    comp_matrix=companion_matrix(poly, MatrixSpace(GF(q),the_n,the_n))
    torus_elements = nonsplit_embedding(comp_matrix, q, B)
    for a,b,c,d in GF(q)^4:
        entries = [0,1,a,0,0,b,0,0,0,1,0,c,0,0,0,0,1,d]
        element = mspace.matrix(entries)
        elements_with_specific_minors = set()
        for m in torus_elements:
            result = (element*m).echelon_form()
            minors = result.minors(the_k)
            if minors[0] == 1 and minors[4]==-1 and minors[5]==1 and minors[9]==1 and minors[12]==1 and minors[14]==1:
                elements_with_specific_minors.add(result)    
        if len(elements_with_specific_minors) != 1:
            return False
    return True

######################################################################
# Normal elements
######################################################################
normal_elems = []
for a,b,c,d,e,f in GF(the_q)^the_n:
    if a!=0 or b!=0 or c!=0 or d!=0 or e!=0 or f!=0:
        generating_poly = a*(x^5).polynomial(GF(the_q))+b*(x^4).polynomial(GF(the_q))+c*(x^3).polynomial(GF(the_q))+d*(x^2).polynomial(GF(the_q))+e*(x^1).polynomial(GF(the_q))+f*(x^0).polynomial(GF(the_q))
        powers = [the_A(generating_poly^(the_q^i)).lift() for i in range(the_n)]
        matrix = basis_to_change_matrix(powers, the_q)
        if matrix.det() != 0:
            normal_elems.append(generating_poly)
            

for p in normal_elems:
    print("Normal:", p)
print(len(normal_elems))


######################################################################
#  Putting it all together
######################################################################

works = []
for poly in normal_elems:
    powers = [the_A(poly^(the_q^i)).lift() for i in range(the_n)]
    matrix = basis_to_change_matrix(powers,the_q)
    result345 = threefourfive_test((x^6+x+1).polynomial(GF(the_q)), the_q, matrix)
    result245 = twofourfive_test((x^6+x+1).polynomial(GF(the_q)), the_q, matrix)
    if result345 and result245:
        print("Normal element that's working:", poly)
        print(matrix)
        print()
        works.append(poly)

print(len(works))

Normal: x^5
Normal: x^5 + x^4
Normal: x^5 + x^3
Normal: x^5 + x^4 + x^3
Normal: x^5 + x^2
Normal: x^5 + x^4 + x^3 + x^2
Normal: x^5 + x^4 + x
Normal: x^5 + x^3 + x
Normal: x^5 + x^2 + x
Normal: x^5 + x^4 + x^2 + x
Normal: x^5 + x^3 + x^2 + x
Normal: x^5 + x^4 + x^3 + x^2 + x
Normal: x^5 + 1
Normal: x^5 + x^4 + 1
Normal: x^5 + x^3 + 1
Normal: x^5 + x^4 + x^3 + 1
Normal: x^5 + x^2 + 1
Normal: x^5 + x^4 + x^3 + x^2 + 1
Normal: x^5 + x^4 + x + 1
Normal: x^5 + x^3 + x + 1
Normal: x^5 + x^2 + x + 1
Normal: x^5 + x^4 + x^2 + x + 1
Normal: x^5 + x^3 + x^2 + x + 1
Normal: x^5 + x^4 + x^3 + x^2 + x + 1
24
0
